In [1]:
from xdsl.dialects.builtin import *
from xdsl.printer import Printer
from devito.ir.ietxdsl import *
from xdsl.dialects.builtin import ModuleOp

In [5]:
from devito import Grid, TimeFunction, Eq, Operator

In [12]:
from devito.ir.iet.visitors import Visitor

In [3]:
def test_example():
    ctx = MLContext()
    Builtin(ctx)
    iet = IET(ctx)

    mod = ModuleOp.from_region_or_ops([
        Callable.get("kernel", ["u"], Block.from_callable([iet.i32], lambda u: [
            Iteration.get(["affine", "sequential"], ("time_m", "time_M", "1"),
                          Block.from_callable([iet.i32, iet.i32, iet.i32], lambda time, t0, t1: [
                Iteration.get(["affine", "parallel", "skewable"], ("x_m", "x_M", "1"),
                              Block.from_callable([iet.i32], lambda x: [
                    Iteration.get(["affine", "parallel", "skewable", "vector-dim"], ("y_m", "y_M", "1"),
                                  Block.from_callable([iet.i32], lambda y: [
                        cst1    := Constant.get(1),
                        x1      := Addi.get(x, cst1),
                        y1      := Addi.get(y, cst1),
                        ut0     := Idx.get(u, t0),
                        ut0x1   := Idx.get(ut0, x1),
                        ut0x1y1 := Idx.get(ut0x1, y1),
                        rhs     := Addi.get(ut0x1y1, cst1),
                        ut1     := Idx.get(u, t1),
                        ut1x1   := Idx.get(ut1, x1),
                        lhs     := Idx.get(ut1x1, y1),
                        Assign.build([lhs, rhs])
                    ]))
                ]))
            ]))
        ]))
    ])

    printer = Printer()
    printer.print_op(mod)

test_example()

module() {
  iet.callable() ["callable_name" = "kernel", "parameters" = ["u"]] {
  ^0(%0 : !i32):
    iet.iteration() ["limits" = ["time_m", "time_M", "1"], "properties" = ["affine", "sequential"]] {
    ^1(%1 : !i32, %2 : !i32, %3 : !i32):
      iet.iteration() ["limits" = ["x_m", "x_M", "1"], "properties" = ["affine", "parallel", "skewable"]] {
      ^2(%4 : !i32):
        iet.iteration() ["limits" = ["y_m", "y_M", "1"], "properties" = ["affine", "parallel", "skewable", "vector-dim"]] {
        ^3(%5 : !i32):
          %6 : !i32 = iet.constant() ["value" = 1 : !i32]
          %7 : !i32 = iet.addi(%4 : !i32, %6 : !i32)
          %8 : !i32 = iet.addi(%5 : !i32, %6 : !i32)
          %9 : !i32 = iet.idx(%0 : !i32, %2 : !i32)
          %10 : !i32 = iet.idx(%9 : !i32, %7 : !i32)
          %11 : !i32 = iet.idx(%10 : !i32, %8 : !i32)
          %12 : !i32 = iet.addi(%11 : !i32, %6 : !i32)
          %13 : !i32 = iet.idx(%0 : !i32, %3 : !i32)
          %14 : !i32 = iet.idx(%13 : !i32, %7 : !i32

In [11]:
grid = Grid(shape = (3, 3))
u = TimeFunction(name='u', grid=grid)
eq = Eq(u.forward, u + 1)
op = Operator([eq])
print(op.view)

<Callable Kernel>
  <CallableBody <allocs=0, casts=1, maps=0> <unmaps=0, frees=0>>
    <List (4, 0, 0)>
      <C.Comment /* Flush denormal numbers to zero in hardware */>
      <C.Statement _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);>
      <C.Statement _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);>
      <C.Line >


    <List (0, 1, 0)>

      <[affine,sequential] Iteration time::time::(time_m, time_M, 1)>
        <Section (section0)>

          <TimedList (1, 1, 1)>
            <[affine,parallel,parallel=,skewable] Iteration x::x::(x_m, x_M, 1)>
              <[affine,parallel,parallel=,skewable,vector-dim] Iteration y::y::(y_m, y_M, 1)>
                <ExpressionBundle (1)>

                  <Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 1>





In [25]:
import devito.ir.iet.nodes as nodes

In [26]:
def myVisit(node):
    if isinstance(node, nodes.CallableBody):
        print(node)
        return 
    
    raise TypeError(f'Unsupported type of node: {type(node)}')

myVisit(op.body)

float (*restrict u)[u_vec->size[1]][u_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]]) u_vec->data;

/* Flush denormal numbers to zero in hardware */
_MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
_MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  /* Begin section0 */
  START_TIMER(section0)
  for (int x = x_m; x <= x_M; x += 1)
  {
    #pragma omp simd aligned(u:32)
    for (int y = y_m; y <= y_M; y += 1)
    {
      u[t1][x + 1][y + 1] = u[t0][x + 1][y + 1] + 1;
    }
  }
  STOP_TIMER(section0,timers)
  /* End section0 */
}
